add volume thresholds and only trade when the underlying isn't going exponential against the spread positions Also only run a naive backtest

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from ib_insync import *
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display

# --- Parameters ---
symbol1 = 'GOOGL'
symbol2 = symbol1
lastTradeDateOrContractMonth1 = '20250725'
lastTradeDateOrContractMonth2 = lastTradeDateOrContractMonth1
strike1 = 185.0
strike2 = 175.0
right1 = 'P'
right2 = right1
barSize = '5 mins'
duration = '7 D'
window = 20
num_std = 2

# --- Connect to IBKR ---
ib = IB()
await ib.connectAsync('127.0.0.1', 7497, clientId=np.random.randint(1000, 9999))

# --- Define and qualify option contracts ---
option1 = Option(
    symbol=symbol1,
    lastTradeDateOrContractMonth=lastTradeDateOrContractMonth1,
    strike=strike1,
    right=right1,
    exchange='SMART',
    currency='USD'
)
option2 = Option(
    symbol=symbol2,
    lastTradeDateOrContractMonth=lastTradeDateOrContractMonth2,
    strike=strike2,
    right=right2,
    exchange='SMART',
    currency='USD'
)

await ib.qualifyContractsAsync(option1, option2)

# --- Subscribe to live bars ---
bars1 = ib.reqHistoricalData(
    option1, endDateTime='', durationStr=duration, barSizeSetting=barSize,
    whatToShow='TRADES', useRTH=True, formatDate=1, keepUpToDate=True
)
bars2 = ib.reqHistoricalData(
    option2, endDateTime='', durationStr=duration, barSizeSetting=barSize,
    whatToShow='TRADES', useRTH=True, formatDate=1, keepUpToDate=True
)

# --- Create Plotly FigureWidget ---
fig = go.FigureWidget()
fig.add_scatter(name='Spread', line=dict(color='black'))
fig.add_scatter(name='BB Upper', line=dict(color='green', dash='dash'))
fig.add_scatter(name='BB Lower', line=dict(color='red', dash='dash'))
fig.add_scatter(name='BB Middle', line=dict(color='orange', dash='dot'))
fig.add_scatter(name='Buy Signal', mode='markers', marker=dict(symbol='triangle-up', color='green', size=12))
fig.add_scatter(name='Sell Signal', mode='markers', marker=dict(symbol='triangle-down', color='red', size=12))
fig.update_layout(
    title='Live Spread with Bollinger Bands and Signals',
    xaxis_title='Time',
    yaxis_title='Spread',
    template='plotly_white',
    width=1000,
    height=500
)
display(fig)

pending_entry = None  # for trade logic

def update_chart_and_logic():
    global pending_entry
    # Convert to DataFrames
    df1 = util.df(bars1).set_index('date').rename(columns={'close': 'Close1'})
    df2 = util.df(bars2).set_index('date').rename(columns={'close': 'Close2'})

    # Fix: use suffixes so all columns are unique (prevents ValueError)
    datafix = df1.join(df2, how='inner', lsuffix='_1', rsuffix='_2')
    datafix['spread'] = datafix['Close1'] - datafix['Close2']

    # Bollinger Bands
    datafix['bb_middle'] = datafix['spread'].rolling(window).mean()
    datafix['bb_std'] = datafix['spread'].rolling(window).std()
    datafix['bb_upper'] = datafix['bb_middle'] + num_std * datafix['bb_std']
    datafix['bb_lower'] = datafix['bb_middle'] - num_std * datafix['bb_std']

    # Signal logic
    datafix['prev_spread'] = datafix['spread'].shift(1)
    datafix['prev_bb_lower'] = datafix['bb_lower'].shift(1)
    datafix['prev_bb_upper'] = datafix['bb_upper'].shift(1)
    datafix['bb_signal'] = 'hold'
    buy_mask = (datafix['prev_spread'] > datafix['prev_bb_lower']) & (datafix['spread'] < datafix['bb_lower'])
    sell_mask = (datafix['prev_spread'] < datafix['prev_bb_upper']) & (datafix['spread'] > datafix['bb_upper'])
    datafix.loc[buy_mask, 'bb_signal'] = 'buy'
    datafix.loc[sell_mask, 'bb_signal'] = 'sell'

    # Trading logic on latest row
    if not datafix.empty:
        last_row = datafix.iloc[-1]
        signal = last_row['bb_signal']
        print(f"New or initial bar at {last_row.name} | Spread: {last_row['spread']:.2f} | Signal: {signal}")

        positions = {p.contract.conId: p.position for p in ib.positions()}
        pos1 = positions.get(option1.conId, 0)
        pos2 = positions.get(option2.conId, 0)
        if pos1 == 1 and pos2 == -1:
            current_position = 'long'
        elif pos1 == -1 and pos2 == 1:
            current_position = 'short'
        else:
            current_position = 'flat'

        if current_position == 'long' and signal == 'sell':
            ib.placeOrder(option1, MarketOrder('SELL', 1))
            ib.placeOrder(option2, MarketOrder('BUY', 1))
            pending_entry = 'short'
        elif current_position == 'short' and signal == 'buy':
            ib.placeOrder(option1, MarketOrder('BUY', 1))
            ib.placeOrder(option2, MarketOrder('SELL', 1))
            pending_entry = 'long'
        elif current_position == 'flat' and pending_entry is not None:
            if pending_entry == 'long':
                ib.placeOrder(option1, MarketOrder('BUY', 1))
                ib.placeOrder(option2, MarketOrder('SELL', 1))
            elif pending_entry == 'short':
                ib.placeOrder(option1, MarketOrder('SELL', 1))
                ib.placeOrder(option2, MarketOrder('BUY', 1))
            pending_entry = None
        elif current_position == 'flat' and pending_entry is None:
            if signal == 'buy':
                ib.placeOrder(option1, MarketOrder('BUY', 1))
                ib.placeOrder(option2, MarketOrder('SELL', 1))
            elif signal == 'sell':
                ib.placeOrder(option1, MarketOrder('SELL', 1))
                ib.placeOrder(option2, MarketOrder('BUY', 1))

    # --- Update plot (last 50 bars) ---
    plot_df = datafix.tail(50)
    buy_signals = plot_df[plot_df['bb_signal'] == 'buy']
    sell_signals = plot_df[plot_df['bb_signal'] == 'sell']
    with fig.batch_update():
        fig.data[0].x = plot_df.index
        fig.data[0].y = plot_df['spread']
        fig.data[1].x = plot_df.index
        fig.data[1].y = plot_df['bb_upper']
        fig.data[2].x = plot_df.index
        fig.data[2].y = plot_df['bb_lower']
        fig.data[3].x = plot_df.index
        fig.data[3].y = plot_df['bb_middle']
        fig.data[4].x = buy_signals.index
        fig.data[4].y = buy_signals['spread']
        fig.data[5].x = sell_signals.index
        fig.data[5].y = sell_signals['spread']

update_chart_and_logic()

def on_new_bar(bars, has_new_bar):
    if has_new_bar:
        update_chart_and_logic()

bars1.updateEvent += on_new_bar
bars2.updateEvent += on_new_bar

print("Live strategy, trading, and chart are running. Awaiting new bars...")
import asyncio
await asyncio.sleep(1e6)


FigureWidget({
    'data': [{'line': {'color': 'black'},
              'name': 'Spread',
              'type': 'scatter',
              'uid': 'aaed337b-b3c2-4a82-810a-5fe9931a952f'},
             {'line': {'color': 'green', 'dash': 'dash'},
              'name': 'BB Upper',
              'type': 'scatter',
              'uid': '53059edd-b168-4f17-90de-ab107db7f441'},
             {'line': {'color': 'red', 'dash': 'dash'},
              'name': 'BB Lower',
              'type': 'scatter',
              'uid': 'e198c6e3-270c-4e81-83da-f0fb775d24ce'},
             {'line': {'color': 'orange', 'dash': 'dot'},
              'name': 'BB Middle',
              'type': 'scatter',
              'uid': '8208d5df-e7bf-4c07-ac34-4a93961ed2b3'},
             {'marker': {'color': 'green', 'size': 12, 'symbol': 'triangle-up'},
              'mode': 'markers',
              'name': 'Buy Signal',
              'type': 'scatter',
              'uid': '014ebb1a-3c4a-4e25-9608-0a3afd636940'},
           

New or initial bar at 2025-07-15 14:45:00-04:00 | Spread: 3.95 | Signal: hold
Live strategy, trading, and chart are running. Awaiting new bars...
